In [1]:
import os  # For fetching environment variables
import sys

# Add the parent directory to the sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import math
import datetime
import time
from eth_utils import to_bytes, to_int
import requests

from pprint import pprint
from aave_client import AaveClient

/Users/cleme/cleyfe/Aave_Functions/aave_functions/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Setup client

RPC = "ARBITRUM"

if RPC == "POLYGON":
    RPC_URL = os.getenv("POLYGON_RPC_URL")
elif RPC == "ETHEREUM":
    RPC_URL = os.getenv("MAINNET_RPC_URL")
elif RPC == "ARBITRUM":
    RPC_URL = os.getenv("ARBITRUM_RPC_URL")
elif RPC == "KOVAN":
    RPC_URL = os.getenv("KOVAN_RPC_URL")
elif RPC == "MUMBAI":
    RPC_URL = os.getenv("MUMBAI_RPC_URL")
else:
    RPC_URL = None  # or handle the case when the RPC does not match any known value

# Define a dictionary to map RPC to the appropriate keyword argument
rpc_url_args = {
    "POLYGON": "POLYGON_RPC_URL",
    "ETHEREUM": "MAINNET_RPC_URL",
    "ARBITRUM": "ARBITRUM_RPC_URL",
    "KOVAN": "KOVAN_RPC_URL",
    "MUMBAI": "MUMBAI_RPC_URL"
}

# Determine the correct argument to pass based on the RPC value
rpc_arg = {rpc_url_args[RPC]: RPC_URL}

In [4]:
# Initialize Client:
aave_client = AaveClient(WALLET_ADDRESS=os.getenv('WALLET_ADDRESS'),
                                PRIVATE_WALLET_KEY=os.getenv('PRIVATE_WALLET_KEY'),
                                GAS_STRATEGY="medium",
                                **rpc_arg)

# Get the lending pool smart contract:
lending_pool = aave_client.get_lending_pool()
print(lending_pool.address)

0x794a61358D6845594F94dc1DB02A252b5b4814aD


In [5]:
""" ------------------------------------------- Testing Cases ------------------------------------------------ """

available_borrow_base, total_debt_base, total_collateral_base = aave_client.get_user_data(lending_pool)
available_borrow_base, total_debt_base, total_collateral_base

(6294276596.0, 34598.0, 9990970149.0)

In [6]:
''' ----------------- Call protocol functions ----------------- '''
function_name = "getAllReservesTokens"
asset_address = '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'
user_address = os.getenv('WALLET_ADDRESS')

# Example usage for getAllReservesTokens which requires no parameters
assets = aave_client.get_protocol_data("getAllReservesTokens")
print(assets)

# Example usage for getReserveConfigurationData which requires an asset address parameter
#result = aave_client.get_protocol_data("getReserveConfigurationData", asset_address)
#print(result)

# Example usage for getUserReserveData which requires asset and user address parameters
user_reserves = aave_client.get_protocol_data("getUserReserveData", asset_address, user_address)
print(user_reserves)

[('DAI', '0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1'), ('LINK', '0xf97f4df75117a78c1A5a0DBb814Af92458539FB4'), ('USDC', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'), ('WBTC', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f'), ('WETH', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'), ('USDT', '0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9'), ('AAVE', '0xba5DdD1f9d7F570dc94a51479a000E3BCE967196'), ('EURS', '0xD22a58f79e9481D1a88e00c343885A588b34b68B'), ('wstETH', '0x5979D7b546E38E414F7E9822514be443A4800529'), ('MAI', '0x3F56e0c36d275367b8C502090EDF38289b3dEa0d'), ('rETH', '0xEC70Dcb4A1EFa46b8F2D97C310C9c4790ba5ffA8'), ('LUSD', '0x93b346b6BC2548dA6A1E7d98E9a421B42541425b'), ('USDC', '0xaf88d065e77c8cC2239327C5EDb3A432268e5831'), ('FRAX', '0x17FC002b466eEc40DaE837Fc4bE5c67993ddBd6F'), ('ARB', '0x912CE59144191C1204E64559FE8253a0e49E6548'), ('weETH', '0x35751007a407ca6FEFfE80b3cB397736D2cf4dbe')]
[0, 0, 0, 0, 0, 0, 117773204487015202727041090, 0, False]


In [7]:
''' ----------------- Call pool functions ----------------- '''
#assets = [('DAI', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'), ('LINK', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39'), ('USDC', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'), ('WBTC', '0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6'), ('WETH', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'), ('USDT', '0xc2132D05D31c914a87C6611C10748AEb04B58e8F'), ('AAVE', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B'), ('WMATIC', '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270'), ('CRV', '0x172370d5Cd63279eFa6d502DAB29171933a610AF'), ('SUSHI', '0x0b3F868E0BE5597D5DB7fEB59E1CADBb0fdDa50a'), ('GHST', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7'), ('BAL', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3'), ('DPI', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369'), ('EURS', '0xE111178A87A3BFf0c8d18DECBa5798827539Ae99'), ('jEUR', '0x4e3Decbb3645551B8A19f0eA1678079FCB33fB4c'), ('EURA', '0xE0B52e49357Fd4DAf2c15e02058DCE6BC0057db4'), ('miMATIC', '0xa3Fa99A148fA48D14Ed51d610c367C61876997F1'), ('stMATIC', '0x3A58a54C066FdC0f2D55FC9C89F0415C92eBf3C4'), ('MaticX', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6'), ('wstETH', '0x03b54A6e9a984069379fae1a4fC4dBAE93B3bCCD'), ('USDC', '0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359')]
assets_symbols = ['DAI', 'USDC', 'USDT']
user_address = os.getenv('WALLET_ADDRESS')

filtered_assets = [asset for asset in assets if asset[0] in assets_symbols]

df = pd.DataFrame(columns=['symbol', 'currentLiquidityRate', 'currentVariableBorrowRate', 'lastUpdateTimestamp'])

# Iterate over the filtered assets and call the desired functions
for symbol, asset_address in filtered_assets:
    # Call getReserveData for each asset
    reserve_data = aave_client.get_pool_data(lending_pool, "getReserveData", asset_address)
    # Extract the required data
    current_liquidity_rate = reserve_data['currentLiquidityRate'] * 100
    current_variable_borrow_rate = reserve_data['currentVariableBorrowRate'] * 100
    last_update_timestamp = pd.to_datetime(reserve_data['lastUpdateTimestamp'], unit='s')
    
    # Append the data to the DataFrame
    df.loc[asset_address] = [symbol, current_liquidity_rate, current_variable_borrow_rate, last_update_timestamp]

df

,symbol,currentLiquidityRate,currentVariableBorrowRate,lastUpdateTimestamp
0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1,DAI,10.636377,16.238505,2024-06-06 11:44:18
0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8,USDC,8.300846,16.922630,2024-06-06 11:24:04
0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9,USDT,11.777320,14.767760,2024-06-06 11:45:55
0xaf88d065e77c8cC2239327C5EDb3A432268e5831,USDC,15.068525,18.288537,2024-06-06 11:45:32


In [8]:
''' ----------------- Spread between largest supply rate and smallest borrow rate ----------------- '''
spread = df['currentLiquidityRate'].max() - df['currentVariableBorrowRate'].min()
spread

0.30076560691157006

In [9]:
'''
Maximum leverage with x LTV if we borrow until the borrowing amount becomes less than 10% of the initial capital

Bn < 0.1 * C0
0.9^n < 0.1
n>21.85
hence 22 loops
'''
LTV = 0.7
initial_collateral = 99 # e.g. $100
stop_condition = 0.4

number_of_loops = math.ceil(math.log(stop_condition) / math.log(LTV))
print("number_of_loops: ", number_of_loops)

#total_collateral = 1000 * (1 / (1-0.9))
total_collateral = initial_collateral * ((1 - 0.9**number_of_loops) / 0.1)
leverage = total_collateral / initial_collateral

print("total_collateral", total_collateral, "leverage", leverage)

number_of_loops:  3
total_collateral 268.2899999999999 leverage 2.709999999999999


In [10]:
''' ------------------------ Final APY ------------------'''
apy = ( df['currentLiquidityRate'].max() * initial_collateral + (total_collateral - initial_collateral) * max(spread,0) ) / initial_collateral
apy

15.582834524129064

In [11]:
# user account data across the pools 
user_account_data = aave_client.get_pool_data(lending_pool, "getUserAccountData", user_address)
print(user_account_data)
total_supply = user_account_data['totalCollateralBase']
total_borrow = user_account_data['totalDebtBase']

{'totalCollateralBase': 99.9097025, 'totalDebtBase': 0.00034598, 'availableBorrowsBase': 62.9427666, 'currentLiquidationThreshold': 7700, 'ltv': 6300, 'healthFactor': 222355254436672640036996}


In [12]:
# Call getUserReserveData 
user_reserve_data = aave_client.get_pool_data(lending_pool, "getUserConfiguration", user_address)
print(f"User reserve data: {user_reserve_data}")

User reserve data: (18,)


In [13]:
reserves_assets_list = aave_client.get_pool_data(lending_pool, "getReservesList")


In [14]:
# Call getUserReserveData
user_reserve_data = aave_client.get_pool_data(lending_pool, "getUserConfiguration", user_address)

def interpret_user_reserve_data(user_reserve_data):
    # Extract the relevant value from the tuple
    user_reserve_value = user_reserve_data[0]

    # Convert the user_reserve_value to binary representation
    binary_data = bin(user_reserve_value)[2:]

    # Pad the binary data with leading zeros to ensure it has a length of 32 bits
    binary_data = binary_data.zfill(32)

    # Split the binary data into pairs of bits
    bit_pairs = [binary_data[i:i+2] for i in range(0, len(binary_data), 2)]

    # Initialize empty lists to store the indices of borrowed and collateral assets
    borrowed_indices = []
    collateral_indices = []

    # Iterate over the bit pairs, starting from the right
    for i in range(len(bit_pairs)-1, -1, -1):
        pair = bit_pairs[i]
        asset_index = len(bit_pairs) - i - 1

        if pair == "01":
            borrowed_indices.append(asset_index)
        elif pair == "11":
            collateral_indices.append(asset_index)
            borrowed_indices.append(asset_index)
        elif pair == "10":
            collateral_indices.append(asset_index)

    return borrowed_indices, collateral_indices

borrowed_indices, collateral_indices = interpret_user_reserve_data(user_reserve_data)

In [21]:
# verify that borrowed and collateral assets are unique
if len(borrowed_indices) > 1:
    print("More than 1 asset are being borrowed")

if len(collateral_indices) > 1:
    print("More than 1 asset are being used as collateral")

print("Supplied asset is: ", reserves_assets_list[collateral_indices[0]])
print("Borrowed asset is: ", reserves_assets_list[borrowed_indices[0]])

Supplied asset is:  0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1
Borrowed asset is:  0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8


In [22]:
supply_asset = reserves_assets_list[collateral_indices[0]]
supply_reserve_asset = next((token for token in aave_client.active_network.aave_tokens if token.address == supply_asset), None)
supply_reserves = aave_client.get_protocol_data("getUserReserveData", supply_asset, user_address)

borrow_asset = reserves_assets_list[borrowed_indices[0]]
borrow_reserve_asset = next((token for token in aave_client.active_network.aave_tokens if token.address == borrow_asset), None)
borrow_reserves = aave_client.get_protocol_data("getUserReserveData", borrow_asset, user_address)


print("Supplied asset address is", supply_asset, "for a total of", supply_reserves[0])
print("Borrowed asset address is", borrow_asset, "for a total of", borrow_reserves[2])

Supplied asset address is 0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1 for a total of 99963950769135412048
Borrowed asset address is 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8 for a total of 69975064


In [23]:
borrow_asset_wallet_balance = aave_client.get_wallet_balance_data("balanceOf", user_address, borrow_asset)

print("Wallet has", aave_client.convert_from_decimal_units(borrow_reserve_asset, borrow_asset_wallet_balance), "balance of the borrowed assets, while the debt is", aave_client.convert_from_decimal_units(borrow_reserve_asset, borrow_reserves[2]))

diff_borrow = borrow_asset_wallet_balance - borrow_reserves[2]
if diff_borrow < 0:
    print("The debt can't be repaid with wallet proceeds only, need to sell collateral")
else:
    print("The debt can be repaid using wallet balance")

Wallet has 69.97474 balance of the borrowed assets, while the debt is 69.975064
The debt can't be repaid with wallet proceeds only, need to sell collateral


In [19]:
"""Deposit tokens"""
deposit_token = aave_client.active_network.USDT
reserve_token = next((token for token in aave_client.active_network.aave_tokens if token.address == deposit_token), None)

deposit_hash = aave_client.deposit(deposit_token=reserve_token, deposit_amount=initial_collateral)
print("Transaction Hash:", deposit_hash)

Approving transaction to deposit 99 of USDT to Aave...
Approved 99000000 of 0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9 for contract 0x794a61358D6845594F94dc1DB02A252b5b4814aD
Depositing 99 of USDT to Aave...
Awaiting transaction receipt for transaction hash: 0xfc4957acfb401cd8a7e3b6e487b4102f82174c2e3a0aaea134c6cee9c5cfb9ec (timeout = 300 seconds)
Successfully deposited 99 of USDT
Transaction Hash: AaveTrade(hash='0xfc4957acfb401cd8a7e3b6e487b4102f82174c2e3a0aaea134c6cee9c5cfb9ec', timestamp=1717596451, datetime='2024-06-05 16:07:31', contract_address=None, from_address='0x64b27E5b4cB9f4D6743Ea074de813f1401e36a5f', to_address='0x794a61358D6845594F94dc1DB02A252b5b4814aD', gas_price=Decimal('0.00000483911'), asset_symbol='USDT', asset_address='0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9', asset_amount=99, asset_amount_decimal_units=99000000, interest_rate_mode=None, operation='Deposit')


In [20]:
"""withdraw tokens"""
#withdraw_token = supply_asset
#reserve_token = next((token for token in aave_client.active_network.aave_tokens if token.address == withdraw_token), None)
#withdraw_transaction_receipt = aave_client.withdraw(withdraw_token=supply_reserve_asset, withdraw_amount=total_supply)

withdraw_amount = 1.01 * aave_client.convert_from_decimal_units(borrow_reserve_asset, abs(diff_borrow)) # with 1% margin for rounding effects
withdraw_transaction_receipt = aave_client.withdraw(withdraw_token=supply_reserve_asset, withdraw_amount=withdraw_amount)
print("Transaction Hash:", withdraw_transaction_receipt)


Approving transaction to withdraw 0.016567030000000000 of DAI from Aave...
Approved 16567030000000000 of 0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1 for contract 0x794a61358D6845594F94dc1DB02A252b5b4814aD
Withdrawing 0.01656703 of DAI from Aave...
Awaiting transaction receipt for transaction hash: 0xcb3ddb4bf84b8714df15f1bac3f3466e10565331157f35d977e51158551404db (timeout = 300 seconds)
Successfully withdrew 0.016567030000000000 of DAI from Aave
Transaction Hash: AaveTrade(hash='0xcb3ddb4bf84b8714df15f1bac3f3466e10565331157f35d977e51158551404db', timestamp=1717664344, datetime='2024-06-06 10:59:04', contract_address=None, from_address='0x64b27E5b4cB9f4D6743Ea074de813f1401e36a5f', to_address='0x794a61358D6845594F94dc1DB02A252b5b4814aD', gas_price=Decimal('0.00000559206'), asset_symbol='DAI', asset_address='0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1', asset_amount=0.01656703, asset_amount_decimal_units=16567030000000000, interest_rate_mode=None, operation='Withdraw')


In [21]:
"""swap tokens"""
# Fetch the reserve tokens
aave_client.active_network.aave_tokens = aave_client.active_network.fetch_aave_tokens()

# Identify the swap from and swap to tokens
swap_from_token = next((token for token in aave_client.active_network.aave_tokens if token.address == supply_asset), None)
swap_to_token = next((token for token in aave_client.active_network.aave_tokens if token.address == borrow_asset), None)

if not swap_from_token or not swap_to_token:
    raise ValueError("Tokens not found in the Aave reserve tokens list.")

# Define the amount to swap and minimum amount to receive
amount_to_swap = withdraw_amount  # Example: 100 DAI

# Perform the swap
try:
    receipt = aave_client.swap(
        swap_from_token=swap_from_token,
        swap_to_token=swap_to_token,
        amount_to_swap=amount_to_swap,
    )
    print(f"Swap and deposit transaction completed. Receipt: {receipt}")
except Exception as e:
    print(f"Error executing swap: {e}")


Approved 16567030000000000 of 0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1 for contract 0x216B4B4Ba9F3e719726886d34a177484278Bfcae
Error executing swap: Could not execute swap - Error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: address 0x64b27E5b4cB9f4D6743Ea074de813f1401e36a5f have 72534621307257 want 127860992000000'}


In [24]:
"""repay tokens"""
#repay_token = borrow_asset
#reserve_token = next((token for token in aave_client.active_network.aave_tokens if token.address == repay_token), None)
repay_amount = aave_client.convert_from_decimal_units(borrow_reserve_asset, borrow_asset_wallet_balance)
repay_transaction_receipt = aave_client.repay(lending_pool_contract=lending_pool, repay_amount=repay_amount, repay_asset=borrow_reserve_asset)
print("Transaction Hash:", repay_transaction_receipt)


Time to repay...
Approving transaction to repay 69.97474 of USDCE to Aave...
Approved 69974740 of 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8 for contract 0x794a61358D6845594F94dc1DB02A252b5b4814aD
Transaction approved!
Repaying 69.97474 of USDCE...
Awaiting transaction receipt for transaction hash: 0x98fb37ad263d1a622d8a57f5aba39767add6c692e870d8f5851f2959d682ca29 (timeout = 300 seconds)
Repaid 69.97474 USDCE  |  36698.000000000000000000 ETH worth of debt remaining.
Transaction Hash: AaveTrade(hash='0x98fb37ad263d1a622d8a57f5aba39767add6c692e870d8f5851f2959d682ca29', timestamp=1717667586, datetime='2024-06-06 11:53:06', contract_address=None, from_address='0x64b27E5b4cB9f4D6743Ea074de813f1401e36a5f', to_address='0x794a61358D6845594F94dc1DB02A252b5b4814aD', gas_price=Decimal('0.00000510074'), asset_symbol='USDCE', asset_address='0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8', asset_amount=69.97474, asset_amount_decimal_units=69974740, interest_rate_mode='variable', operation='Repay')


In [20]:
"""borrow tokens"""
borrow_amount = LTV * aave_client.convert_from_decimal_units(supply_reserve_asset, supply_reserves[0])
borrow_transaction_receipt = aave_client.borrow(lending_pool_contract=lending_pool, borrow_amount=borrow_amount, borrow_asset=borrow_reserve_asset)
print("Transaction Hash:", borrow_transaction_receipt)


Transaction Hash: AaveTrade(hash='0xbd96dff10bcb72549ff29392deb8f0ce48213fb192a8e516f27a5e6c0586926a', timestamp=1717667401, datetime='2024-06-06 11:50:01', contract_address=None, from_address='0x64b27E5b4cB9f4D6743Ea074de813f1401e36a5f', to_address='0x794a61358D6845594F94dc1DB02A252b5b4814aD', gas_price=Decimal('0.00000470395'), asset_symbol='USDCE', asset_address='0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8', asset_amount=69.97469168492466, asset_amount_decimal_units=69974691, interest_rate_mode='variable', operation='Borrow')
